In [ ]:
import pandas as pd
import asyncio
import websockets
import json
import os
from datetime import datetime, timedelta
import nest_asyncio

nest_asyncio.apply()

FINNHUB_API_KEY = "d056jb9r01qoigrsmf5gd056jb9r01qoigrsmf60"

# S&P 500 tickers (limit to first 5 for demo)
url = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv"
tickers = pd.read_csv(url)['Symbol'].tolist()

# WebSocket endpoint
FINNHUB_WS_URL = f"wss://ws.finnhub.io?token={FINNHUB_API_KEY}"

# In-memory storage
df_all_data = pd.DataFrame()

# Create directory
os.makedirs("dati_stream", exist_ok=True)

async def save_data_periodically(interval_sec=60):
    global df_all_data
    while True:
        await asyncio.sleep(interval_sec)
        if not df_all_data.empty:
            timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
            path = f"dati_stream/finnhub_stream_{timestamp}.csv"
            df_all_data.to_csv(path, index=False)
            print(f"💾 Salvato {len(df_all_data)} righe in {path}")

async def stream_finnhub(duration_seconds=60):
    global df_all_data

    async with websockets.connect(FINNHUB_WS_URL) as ws:
        # Subscribe to tickers
        for symbol in tickers:
            await ws.send(json.dumps({"type": "subscribe", "symbol": symbol}))
            print(f"📡 Subscribed to {symbol}")

        # Start saving data
        save_task = asyncio.create_task(save_data_periodically(30))

        end_time = datetime.utcnow() + timedelta(seconds=duration_seconds)

        try:
            while datetime.utcnow() < end_time:
                message = await ws.recv()
                data = json.loads(message)

                if data.get("type") == "trade":
                    for trade in data.get("data", []):
                        row = {
                            "Ticker": trade["s"],
                            "Timestamp": pd.to_datetime(trade["t"], unit="ms", utc=True),
                            "Price": trade["p"],
                            "Size": trade["v"]
                        }
                        df_all_data = pd.concat([df_all_data, pd.DataFrame([row])], ignore_index=True)
                        print(df_all_data.tail(1))
        finally:
            await ws.close()
            save_task.cancel()
            print("✅ Streaming finished.")

# Run the stream
await stream_finnhub(duration_seconds=60)


📡 Subscribed to MMM
📡 Subscribed to AOS
📡 Subscribed to ABT
📡 Subscribed to ABBV
📡 Subscribed to ACN
📡 Subscribed to ADBE
📡 Subscribed to AMD
📡 Subscribed to AES
📡 Subscribed to AFL
📡 Subscribed to A
📡 Subscribed to APD
📡 Subscribed to ABNB
📡 Subscribed to AKAM
📡 Subscribed to ALB
📡 Subscribed to ARE
📡 Subscribed to ALGN
📡 Subscribed to ALLE
📡 Subscribed to LNT
📡 Subscribed to ALL
📡 Subscribed to GOOGL
📡 Subscribed to GOOG
📡 Subscribed to MO
📡 Subscribed to AMZN
📡 Subscribed to AMCR
📡 Subscribed to AEE
📡 Subscribed to AEP
📡 Subscribed to AXP
📡 Subscribed to AIG
📡 Subscribed to AMT
📡 Subscribed to AWK
📡 Subscribed to AMP
📡 Subscribed to AME
📡 Subscribed to AMGN
📡 Subscribed to APH
📡 Subscribed to ADI
📡 Subscribed to ANSS
📡 Subscribed to AON
📡 Subscribed to APA
📡 Subscribed to APO
📡 Subscribed to AAPL
📡 Subscribed to AMAT
📡 Subscribed to APTV
📡 Subscribed to ACGL
📡 Subscribed to ADM
📡 Subscribed to ANET
📡 Subscribed to AJG
📡 Subscribed to AIZ
📡 Subscribed to T
📡 Subscribed to ATO
📡 Subsc

/tmp/ipykernel_53921/3894763523.py:49: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_time = datetime.utcnow() + timedelta(seconds=duration_seconds)
/tmp/ipykernel_53921/3894763523.py:52: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  while datetime.utcnow() < end_time:


  Ticker                        Timestamp   Price  Size
0  GOOGL 2025-04-25 15:37:35.066000+00:00  162.06   100
  Ticker                        Timestamp   Price  Size
1  GOOGL 2025-04-25 15:37:34.744000+00:00  162.06   100


/tmp/ipykernel_53921/3894763523.py:52: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  while datetime.utcnow() < end_time:


  Ticker                        Timestamp   Price  Size
2   GOOG 2025-04-25 15:37:36.144000+00:00  164.14   117
  Ticker                        Timestamp   Price  Size
3   GOOG 2025-04-25 15:37:36.169000+00:00  164.14   200
  Ticker                        Timestamp   Price  Size
4   GOOG 2025-04-25 15:37:36.175000+00:00  164.13   100
  Ticker                        Timestamp   Price  Size
5   GOOG 2025-04-25 15:37:36.175000+00:00  164.13   100
  Ticker                        Timestamp   Price  Size
6  GOOGL 2025-04-25 15:37:37.001000+00:00  162.02   699


...

/tmp/ipykernel_53921/3894763523.py:52: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  while datetime.utcnow() < end_time:


    Ticker                        Timestamp   Price  Size
134  GOOGL 2025-04-25 15:38:32.867000+00:00  161.91   100
    Ticker                        Timestamp  Price  Size
135      T 2025-04-25 15:38:33.247000+00:00  26.69   100
    Ticker                        Timestamp  Price  Size
136      T 2025-04-25 15:38:33.248000+00:00  26.69   500
    Ticker                        Timestamp  Price  Size
137      T 2025-04-25 15:38:33.248000+00:00  26.69   100
    Ticker                        Timestamp  Price  Size
138      T 2025-04-25 15:38:33.248000+00:00  26.69   100


/tmp/ipykernel_53921/3894763523.py:52: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  while datetime.utcnow() < end_time:
/tmp/ipykernel_53921/3894763523.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")


💾 Salvato 139 righe in dati_stream/finnhub_stream_20250425_153834.csv
    Ticker                        Timestamp  Price  Size
139   AMCR 2025-04-25 15:38:34.367000+00:00  9.515   100
✅ Streaming finished.


/tmp/ipykernel_53921/3894763523.py:52: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  while datetime.utcnow() < end_time:
